# Session 2.6: Agents, Function Calling, Tools

## Quellen

- Mermaid Live Editor: https://mermaid.live/edit
- Emerging Patterns in Building AI Agents: https://martinfowler.com/articles/gen-ai-patterns/
    - Hier von AI-Anwendung Code erstellen lassen als Beispiele!

- Bildquelle1 multiagentsystems profit
    - https://x.com/tom_doerr/status/1906206992424693884/photo/1

- OpenAI Agents: https://github.com/openai/openai-agents-python?tab=readme-ov-file / https://openai.github.io/openai-agents-python/
- Agent Patterns by Openai: https://github.com/openai/openai-agents-python/tree/main/examples/agent_patterns

## Praxis

![Multiagentsystems profit](assets/image04_multiagentsystemsprofit.png)

### 🛠️ Imports

In [ ]:
from __future__ import annotations
from aiworkshop_utils.standardlib_imports import os, json, base64, logging, Optional, List, Literal, pprint, glob, asyncio, datetime, date, time, timezone, ZoneInfo, uuid, dataclass
from aiworkshop_utils.thirdparty_imports import AutoTokenizer, load_dotenv, requests, BaseModel, Field, pd, cosine_similarity, plt, np, DataType, MilvusClient, DDGS, rprint
from aiworkshop_utils.custom_utils import show_pretty_json, encode_image
from aiworkshop_utils.jupyter_imports import Markdown, HTML, JSON, display, widgets
from aiworkshop_utils.openai_imports import OpenAI, Agent, Runner, InputGuardrail, GuardrailFunctionOutput, InputGuardrailTripwireTriggered, OpenAIChatCompletionsModel, AsyncOpenAI, set_tracing_disabled, ModelSettings, function_tool, trace, ResponseContentPartDoneEvent, ResponseTextDeltaEvent, RawResponsesStreamEvent, TResponseInputItem, ItemHelpers, MessageOutputItem, RunContextWrapper, input_guardrail, output_guardrail
from aiworkshop_utils import config

### ⚡ Erster Agent mit Agents-SDK

- Was ist ein Agent?
    - Ein Agent ist ein LLM, das mit Instruktionen und Tools konfiguriert ist.
    - "name" -> Name des Agenten
    - "instructions" -> Entwickler Message bzw. Systemprompt
    - "model" -> welches LLM wird benutzt (in "model_settings" kann man z.B. Temperatur ändern)
    - "tools" -> Liste [] an Tools, die das LLM benutzen kann

In [ ]:
endpoint_base = config.OLLAPI_ENDPOINT_BASE

model = config.OMODEL_LLAMA3D2 #model = config.OMODEL_DEEPSEEK Deepseek does not support tools! GEMMA does not support tools!
set_tracing_disabled(True) # um Message "OPENAI_API_KEY is not set, skipping trace export" zu vermeiden

model = OpenAIChatCompletionsModel( 
    model=model,
    openai_client=AsyncOpenAI(base_url=endpoint_base, api_key="fake-key")
)

In [ ]:
first_agent = Agent(
    name="Assistant", 
    instructions="You are a helpful assistant", 
    model=model
    )

In [ ]:
result = await Runner.run(first_agent, "Write a haiku for Eisenstadt, Burgenland, Austria.") # await, weil es in Jupyter einen existing Event Loop gibt
print("-----result")
print(result)
print()
print("-----result.input")
print(result.input)
print()
print("-----result-final_output")
print(result.final_output)


### ⚡ Ein Tool erstellen

- Was ist Function Calling?
    - Function Calling bedeutet, dass LLMs nicht nur Text generieren, sondern gezielt externe Funktionen aufrufen, um an strukturierte Daten zu gelangen oder Aktionen auszuführen (Daten aus DB abfragen, Bestellung aufgeben, ...)
    - Externe Funktionen werden definiert
    - Dem Model wird beschrieben, was die Funktion tut un welche Parameter benötigt werden
    - Wenn eine Nutzeranfrage so eine Aktion erfordert, gibt das LLM nicht die Antwort direkt zurück, sondern ruft die passende Funktion inklusive notwendige Parameter auf
    - Externe Funktion liefert Ergebnis und as Modell gibt die passende Antwort an die/den User:in weiter
    - Z.B. Anfrage „Wie wird das Wetter morgen in Wien?“
    - Modell versteht, es muss Funktion aufrufen:
```
{
  "function": "get_weather_forecast",
  "parameters": {
    "location": "Wien",
    "date": "2025-04-05"
  }
}
```
- Was ist ein Tool?
    - Function Calling ist die technische Fähigkeit eines LLM, eine externe Funktion aktiv aufrufen zu können
    - Ein Tool ist solche externe Funktion oder eine Ressource
    - Ein "Agent" wird instruiert, wann und wie Tools verwendet werden sollen
    - Das Modell entscheidet autonom, welches Tool es verwendet
    - Das Tool liefert eine Antwort in strukturierter Form zurück

In [ ]:
class WeatherData(BaseModel):
    place: str
    # time: datetime
    interval: int
    temperature_2m: float
    wind_speed_10m: float

@function_tool
def get_weather(latitude, longitude) -> WeatherData:
    """
    Fetches the weather for a given location using the Open-Meteo API.
    """
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m,wind_speed_10m"
        f"&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    response = requests.get(url)
    response.raise_for_status()  # Ensure we catch any HTTP errors
    data = response.json()
    return data["current"]

In [ ]:
weather_agent = Agent(
    name="Weather Agent", 
    instructions="You are a helpful weather assisting agent", 
    tools=[get_weather],  # Register the tool with the agent
    model=model
    )

In [ ]:
result = await Runner.run(weather_agent, "What is the weather in Eisenstadt, Burgenland, Austria?")

In [ ]:
print(result.final_output)

In [ ]:
rprint(result)

### 🏋️ **[ÜBUNG_2.6.01]** Multi-Tool-Aufruf

- Erstelle ein oder mehrere weitere Tools neben dem Wetter-API-Tool
- Erstelle einen Agenten, der zwei oder mehr dieser Tools bei seiner Instanzierung als Liste mitnimmt
- Versuche, diesen Agenten mit nur einer User-Message zum Aufrufen mehrerer Tools zu bringen
    - Du kannst tool_choice als Paramter bei der Agentenerstellung auf required setzen mit dem ModelSettings-Objekt
    - um unendliche Loops zu vermeiden, wird nach einem Tool-Aufruf dieser Parameter wieder auf auto gesetzt
```
# Set model settings with tool_choice
model_settings = ModelSettings(
    tool_choice="required",  # or "auto", "none", or a specific tool name
)

agent = Agent(
    name="Weather Agent",
    instructions="You are a helpful weather assisting agent.",
    model=model
    model_settings=model_settings,
    tools=[weather_tool]
)

- Es gibt bei Agent-Erstellung auch den Parameter tool_use_behavior= default -> "run_llm_again", other choice -> "stop_on_first_tool"

### 🔓 **[LÖSUNG_2.7.01]** Multi-Tool Aufruf

#### Minimale Tools und Looping Agent

In [ ]:
class CheckResult(BaseModel):
    is_good: bool

# Agent 1: Text schreiben
writer = Agent(
    name="writer",
    instructions="Schreibe einen sehr kurzen Satz über Technologie.",
    model=model
)

# Agent 2: Qualität prüfen
checker = Agent(
    name="checker",
    instructions="Bewerte, ob der Text klar und verständlich ist. Wenn nicht, sag is_good = False.",
    output_type=CheckResult,
    model=model
)

# Agent 3: Orchestrator mit Schleife
looping_agent = Agent(
    name="looping_agent",
    instructions=(
        "Erzeuge einen Satz über Technologie mit dem 'writer'-Tool.\n"
        "Lass ihn dann vom 'checker'-Tool bewerten.\n"
        "Wenn das Ergebnis nicht gut ist, rufe den 'writer' erneut auf, bis das Ergebnis gut ist."
    ),
    tools=[
        writer.as_tool("generate_text", tool_description="Erzeuge einen Satz."),
        checker.as_tool("check_text", tool_description="Prüfe den Satz."),
    ],
    model=model
)

In [ ]:
result = await Runner.run(looping_agent, "Bitte schreibe einen guten Satz über Technologie.")
print("✅ Final result:\n", result.final_output)

#### Handoffs

In [ ]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    model=model,
    output_type=HomeworkOutput,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=model
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events, people, and context clearly.",
    model=model
)


async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    model=model,
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

In [ ]:
try:
    result = await Runner.run(triage_agent, "I need help for my homework: Who was again the first president of Columbia?")
    print(result.final_output)
except InputGuardrailTripwireTriggered:
    print("It seems your question doesn't relate to homework. Please ask a homework-related question.")

In [ ]:
try:
    result = await Runner.run(triage_agent, "What is life?")
    print(result.final_output)
except InputGuardrailTripwireTriggered:
    print("It seems your question doesn't relate to homework. Please ask a homework-related question.")

In [ ]:
current_date = datetime.now().strftime("%Y-%m")

# 1. Create Internet Search Tool

@function_tool
def get_news_articles(topic):
    print(f"Running DuckDuckGo news search for {topic}...")
    
    # DuckDuckGo search
    ddg_api = DDGS()
    results = ddg_api.text(f"{topic} {current_date}", max_results=5)
    if results:
        news_results = "\n\n".join([f"Title: {result['title']}\nURL: {result['href']}\nDescription: {result['body']}" for result in results])
        print(news_results)
        return news_results
    else:
        return f"Could not find news results for {topic}."
    
# 2. Create AI Agents

# News Agent to fetch news
news_agent = Agent(
    name="News Assistant",
    instructions="You provide the latest news articles for a given topic using DuckDuckGo search.",
    tools=[get_news_articles],
    model=model
)

# Editor Agent to edit news
editor_agent = Agent(
    name="Editor Assistant",
    instructions="Rewrite and give me as news article ready for publishing." 
    "Each News story in separate section."
    "**Do not repeat or duplicate the articles.**",
    model=model
)

# 3. Create workflow

async def run_news_workflow(topic):
    print("Running news Agent workflow...")
    
    # Step 1: Fetch news
    news_response = await Runner.run(
        news_agent,
        f"Get me the news about {topic} on {current_date}"
    )
    
    # Access the content from RunResult object
    raw_news = news_response.final_output
    
    # Step 2: Pass news to editor for final review
    edited_news_response = await Runner.run(
        editor_agent,
        raw_news
    )
    
    # Access the content from RunResult object
    edited_news = edited_news_response.final_output
    
    print("Final news article:")
    print(edited_news)
    
    return "success"

# Example of running the news workflow for a given topic
await run_news_workflow("AI")

### ⚡ Agent Patterns Beispiele von (https://github.com/openai/openai-agents-python/tree/main/examples/agent_patterns)

#### Deterministic

##### Simples Logging Beispiel

In [ ]:
from pathlib import Path

# 1. Create logs directory if it doesn't exist
log_dir = Path("logs")
log_dir.mkdir(exist_ok=True)

# 2. Create a unique log file name with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = log_dir / f"story_flow_{timestamp}.log"

# 3. Configure logging to write to this file
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()  # Also print to Jupyter output cell
    ],
    force=True  # Needed in Jupyter to reset any previous config
)

logging.info(f"Logging to file: {log_file}")

In [ ]:
"""
This example demonstrates a deterministic flow, where each step is performed by an agent.
1. The first agent generates a story outline
2. We feed the outline into the second agent
3. The second agent checks if the outline is good quality and if it is a scifi story
4. If the outline is not good quality or not a scifi story, we stop here
5. If the outline is good quality and a scifi story, we feed the outline into the third agent
6. The third agent writes the story
"""

# Define agents
story_outline_agent = Agent(
    name="story_outline_agent",
    instructions="Generate a very short story outline based on the user's input.",
    model=model
)

class OutlineCheckerOutput(BaseModel):
    good_quality: bool
    is_scifi: bool

outline_checker_agent = Agent(
    name="outline_checker_agent",
    instructions="Read the given story outline, and judge the quality. Also, determine if it is a scifi story.",
    output_type=OutlineCheckerOutput,
    model=model
)

story_agent = Agent(
    name="story_agent",
    instructions="Write a short story based on the given outline.",
    output_type=str,
    model=model
)

# Async function to run the workflow
async def run_story_flow():
    input_prompt = input("What kind of story do you want? ")
    logging.info("Received user input for story type.")

    with trace("Deterministic story flow"):
        # 1. Generate an outline
        logging.info("Invoking story_outline_agent.")
        outline_result = await Runner.run(
            story_outline_agent,
            input_prompt,
        )
        logging.info(f"Outline generated: {outline_result.final_output}")

        # 2. Check the outline
        logging.info("Invoking outline_checker_agent.")
        outline_checker_result = await Runner.run(
            outline_checker_agent,
            outline_result.final_output,
        )

        # 3. Gate logic
        checker_output = outline_checker_result.final_output
        if not checker_output.good_quality:
            logging.warning("Outline is not good quality. Stopping workflow.")
            return
        if not checker_output.is_scifi:
            logging.warning("Outline is not a scifi story. Stopping workflow.")
            return

        logging.info("Outline is good quality and a scifi story. Proceeding to story generation.")

        # 4. Write the story
        logging.info("Invoking story_agent.")
        story_result = await Runner.run(
            story_agent,
            outline_result.final_output,
        )
        logging.info(f"Story generated: {story_result.final_output}")

# Run the async function in Jupyter
await run_story_flow()

#### Handoffs and Routing

In [ ]:
# Define agents
french_agent = Agent(
    name="french_agent",
    instructions="You only speak French",
    model=model
)

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You only speak Spanish",
    model=model
)

english_agent = Agent(
    name="english_agent",
    instructions="You only speak English",
    model=model
)

triage_agent = Agent(
    name="triage_agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[french_agent, spanish_agent, english_agent],
    model=model
)

# Conversation ID for tracing
conversation_id = str(uuid.uuid4().hex[:16])

# Async function
async def run_routing_flow():
    msg = input("Hi! We speak French, Spanish and English. How can I help? ")
    agent = triage_agent
    inputs: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

    while True:
        with trace("Routing example", group_id=conversation_id):
            print(f"\n🧠 Current agent: {agent.name}")
            result = Runner.run_streamed(
                agent,
                input=inputs,
            )
            response = ""

            async for event in result.stream_events():
                if not isinstance(event, RawResponsesStreamEvent):
                    continue
                data = event.data
                if isinstance(data, ResponseTextDeltaEvent):
                    print(data.delta, end="", flush=True)
                    response += data.delta
                elif isinstance(data, ResponseContentPartDoneEvent):
                    print("\n✅ Done\n")

        inputs = result.to_input_list()
        user_msg = input("Enter a message (or type 'exit' to quit): ")
        if user_msg.lower() == "exit":
            print("👋 Conversation ended.")
            break
        inputs.append({"content": user_msg, "role": "user"})
        agent = result.current_agent  # update agent if handed off

# Run in Jupyter
await run_routing_flow()

#### Agenten als Tools

In [ ]:
# Define translation agents
spanish_agent = Agent(
    name="spanish_agent",
    instructions="You translate the user's message to Spanish",
    handoff_description="An english to spanish translator",
    model=model
)

french_agent = Agent(
    name="french_agent",
    instructions="You translate the user's message to French",
    handoff_description="An english to french translator",
    model=model
)

italian_agent = Agent(
    name="italian_agent",
    instructions="You translate the user's message to Italian",
    handoff_description="An english to italian translator",
    model=model
)

# Orchestrator agent that uses the others as tools
orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate. "
        "If asked for multiple translations, you call the relevant tools in order. "
        "You never translate on your own, you always use the provided tools."
    ),
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
        italian_agent.as_tool(
            tool_name="translate_to_italian",
            tool_description="Translate the user's message to Italian",
        ),
    ],
    model=model
)

# Final processing agent
synthesizer_agent = Agent(
    name="synthesizer_agent",
    instructions="You inspect translations, correct them if needed, and produce a final concatenated response.",
    model=model
)

# Async function for notebook
async def run_translation_toolflow():
    msg = input("Hi! What would you like translated, and to which languages? ")

    with trace("Orchestrator evaluator"):
        # First, orchestration with tool usage
        print("\n🔧 Orchestrator is running with selected tools...\n")
        orchestrator_result = await Runner.run(orchestrator_agent, msg)

        for item in orchestrator_result.new_items:
            if isinstance(item, MessageOutputItem):
                text = ItemHelpers.text_message_output(item)
                if text:
                    print(f"🧩 Translation step:\n{text}\n")

        # Then pass everything to the synthesizer
        print("🧪 Synthesizer is generating final output...\n")
        synthesizer_result = await Runner.run(
            synthesizer_agent, orchestrator_result.to_input_list()
        )

    print(f"\n✅ Final response:\n{synthesizer_result.final_output}")

# Run in notebook
await run_translation_toolflow()

In [ ]:
"""
This example shows the LLM as a judge pattern. The first agent generates an outline for a story.
The second agent judges the outline and provides feedback. We loop until the judge is satisfied
with the outline.
"""

story_outline_generator = Agent(
    name="story_outline_generator",
    instructions=(
        "You generate a very short story outline based on the user's input."
        "If there is any feedback provided, use it to improve the outline."
    ),
    model=model
)

@dataclass
class EvaluationFeedback:
    feedback: str
    score: Literal["pass", "needs_improvement", "fail"]

evaluator = Agent[None](
    name="evaluator",
    instructions=(
        "You evaluate a story outline and decide if it's good enough."
        "If it's not good enough, you provide feedback on what needs to be improved."
        "Never give it a pass on the first try."
    ),
    output_type=EvaluationFeedback,
    model=model,
)

async def main() -> None:
    msg = input("What kind of story would you like to hear? ")
    input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

    latest_outline: str | None = None

    # We'll run the entire workflow in a single trace
    with trace("LLM as a judge"):
        while True:
            story_outline_result = await Runner.run(
                story_outline_generator,
                input_items,
            )

            input_items = story_outline_result.to_input_list()
            latest_outline = ItemHelpers.text_message_outputs(story_outline_result.new_items)
            print("Story outline generated")

            evaluator_result = await Runner.run(evaluator, input_items)
            result: EvaluationFeedback = evaluator_result.final_output

            print(f"Evaluator score: {result.score}")

            if result.score == "pass":
                print("Story outline is good enough, exiting.")
                break

            print("Re-running with feedback")

            input_items.append({"content": f"Feedback: {result.feedback}", "role": "user"})

    print(f"Final story outline: {latest_outline}")

# Run the async function in Jupyter
await main()

#### Parallelization

In [ ]:
"""
This example shows the parallelization pattern. We run the agent three times in parallel, and pick
the best result.
"""

# Add your model reference before running, e.g.:
# model = your_model_instance

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You translate the user's message to Spanish",
    model=model,
)

translation_picker = Agent(
    name="translation_picker",
    instructions="You pick the best Spanish translation from the given options.",
    model=model,
)

async def main():
    msg = input("Hi! Enter a message, and we'll translate it to Spanish.\n\n")

    # Ensure the entire workflow is a single trace
    with trace("Parallel translation"):
        res_1, res_2, res_3 = await asyncio.gather(
            Runner.run(spanish_agent, msg),
            Runner.run(spanish_agent, msg),
            Runner.run(spanish_agent, msg),
        )

        outputs = [
            ItemHelpers.text_message_outputs(res_1.new_items),
            ItemHelpers.text_message_outputs(res_2.new_items),
            ItemHelpers.text_message_outputs(res_3.new_items),
        ]

        translations = "\n\n".join(outputs)
        print(f"\n\nTranslations:\n\n{translations}")

        best_translation = await Runner.run(
            translation_picker,
            f"Input: {msg}\n\nTranslations:\n{translations}",
        )

    print("\n\n-----")
    print(f"Best translation: {best_translation.final_output}")

# Run the async function in Jupyter
await main()

#### Input Guardrail

In [ ]:
"""
This example shows how to use guardrails.

Guardrails are checks that run in parallel to the agent's execution.
They can be used to do things like:
- Check if input messages are off-topic
- Check that output messages don't violate any policies
- Take over control of the agent's execution if an unexpected input is detected

In this example, we'll setup an input guardrail that trips if the user is asking to do math homework.
If the guardrail trips, we'll respond with a refusal message.
"""

# Define your model before running this cell, e.g.:
# model = your_model_instance

### 1. An agent-based guardrail that is triggered if the user is asking to do math homework
class MathHomeworkOutput(BaseModel):
    reasoning: str
    is_math_homework: bool

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
    model=model,
)

@input_guardrail
async def math_guardrail(
    context: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    """This is an input guardrail function, which happens to call an agent to check if the input
    is a math homework question.
    """
    result = await Runner.run(guardrail_agent, input, context=context.context)
    final_output = result.final_output_as(MathHomeworkOutput)

    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=final_output.is_math_homework,
    )

### 2. The run loop
async def main():
    agent = Agent(
        name="Customer support agent",
        instructions="You are a customer support agent. You help customers with their questions.",
        input_guardrails=[math_guardrail],
        model=model,
    )

    input_data: list[TResponseInputItem] = []

    while True:
        user_input = input("Enter a message: ")
        input_data.append(
            {
                "role": "user",
                "content": user_input,
            }
        )

        try:
            result = await Runner.run(agent, input_data)
            print(result.final_output)
            # If the guardrail didn't trigger, we use the result as the input for the next run
            input_data = result.to_input_list()
        except InputGuardrailTripwireTriggered:
            # If the guardrail triggered, we instead add a refusal message to the input
            message = "Sorry, I can't help you with your math homework."
            print(message)
            input_data.append(
                {
                    "role": "assistant",
                    "content": message,
                }
            )

# Run the async function in Jupyter
await main()

#### Output Guardrail

In [ ]:
from __future__ import annotations

import json
from pydantic import BaseModel, Field

from agents import (
    Agent,
    GuardrailFunctionOutput,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    output_guardrail,
)

"""
This example shows how to use output guardrails.

Output guardrails are checks that run on the final output of an agent.
They can be used to do things like:
- Check if the output contains sensitive data
- Check if the output is a valid response to the user's message

In this example, we'll use a (contrived) example where we check if the agent's response contains
a phone number.
"""

# Make sure to define your model beforehand:
# model = your_model_instance

# The agent's output type
class MessageOutput(BaseModel):
    reasoning: str = Field(description="Thoughts on how to respond to the user's message")
    response: str = Field(description="The response to the user's message")
    user_name: str | None = Field(description="The name of the user who sent the message, if known")


@output_guardrail
async def sensitive_data_check(
    context: RunContextWrapper, agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
    phone_number_in_response = "650" in output.response
    phone_number_in_reasoning = "650" in output.reasoning

    return GuardrailFunctionOutput(
        output_info={
            "phone_number_in_response": phone_number_in_response,
            "phone_number_in_reasoning": phone_number_in_reasoning,
        },
        tripwire_triggered=phone_number_in_response or phone_number_in_reasoning,
    )


agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    output_type=MessageOutput,
    output_guardrails=[sensitive_data_check],
    model=model,
)

# Main logic for testing the guardrail behavior
async def main():
    # This should be ok
    await Runner.run(agent, "What's the capital of California?")
    print("First message passed")

    # This should trip the guardrail
    try:
        result = await Runner.run(
            agent, "My phone number is 650-123-4567. Where do you think I live?"
        )
        print(
            f"Guardrail didn't trip - this is unexpected. Output: {json.dumps(result.final_output.model_dump(), indent=2)}"
        )

    except OutputGuardrailTripwireTriggered as e:
        print(f"Guardrail tripped. Info: {e.guardrail_result.output.output_info}")

# Run the async function in Jupyter
await main()